# Evaluation for Assistant API

## Introduction

Dataset chosen is the famous `hotspotqa` which is commonly used to evaluate QA and context understanding. 

This notebook is targeted at following goals:

1. Investigate performance of opensource solutions with `mixtral-7bx8` and `LLMCompiler` as function calling strategy.
2. Compares differences between the above solution and the official OpenAI Assistant API (with gpt-3.5-turbo).   


In [1]:
!pip install datasets numpy langchain

In [2]:
%reload_ext autoreload
%autoreload 2

## Prepare dataset

Only hard level questions in [validation split](https://huggingface.co/datasets/scholarly-shadows-syndicate/hotpotqa_with_qa_gpt35/viewer/default/validation) is used in this notebook. 

In [40]:
from datasets import load_dataset

dataset = load_dataset("scholarly-shadows-syndicate/hotpotqa_with_qa_gpt35", split="validation", streaming=True).filter(lambda x: x["level"] == "hard")


## Benchmark runner

* `BenchmarkRunner.run`: load validation dataset and run the QA task, and then save the result to `output_file_path`.
* `Benchmarkrunner.get_metrics`: load runner result from `output_file_path` and calculate metric data.

Only one search tool based on TAVILY API is used during this test and I borrow it from langchain. So make sure that `TAVILY_API_KEY` is set in env variables.

In [46]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain_core.utils.function_calling import convert_to_openai_function

tavily_tool = TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(), max_results=5)

print(convert_to_openai_function(tavily_tool))

result = tavily_tool.invoke("country with most populations")
search_result = "\n".join([item["content"] for item in result])

search_result



{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query']}}


"Countries in the world by population (2024) This list includes both countries and dependent territories. Data based on the latest United Nations Population Division estimates. Click on the name of the country or dependency for current estimates (live population clock), historical data, and projected figures. Fert.\nThe five most populous countries in 2022 are China, India, followed by the European Union (which is not a country), the United States, the island nation of Indonesia, and Pakistan. The Smallest Countries. Among the smallest countries in the world in terms of population are the island nations in the Caribbean and the Southern Pacific Ocean ...\nList of countries by population (United Nations) This is a list of countries and other inhabited territories of the world by total population, based on estimates published by the United Nations in the 2022 revision of World Population Prospects. It presents population estimates from 1950 to the present. [2]\nHowever, a number of count

In [67]:
from openai import OpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
import json
import os
import re
import string
import logging
import numpy as np
import time
from datasets import load_dataset


logging.basicConfig(level=logging.INFO)


def normalize_answer(s):
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False


def compare_answer(answer: str, label: str):
    """Compare the answer (from Agent) and label (GT).
    Label can be either a string or a number.
    If label is a number, we allow 10% margin.
    Otherwise, we do the best-effort string matching.
    """
    if answer is None:
        return False

    # see if label is a number, e.g. "1.0" or "1"
    if is_number(label):
        label = float(label)
        # try cast answer to float and return false if it fails
        try:
            answer = float(answer)
        except:
            return False
        # allow 10% margin
        if label * 0.9 < answer < label * 1.1:
            return True
        else:
            return False

    else:
        label = normalize_answer(label)
        answer = normalize_answer(answer)
        return answer == label


class BenchmarkRunner:
    
    thread_history = []
    
    def __init__(self, client: OpenAI, model_name: str, output_file_path: str = "output/hotqa_result.json"):
        super().__init__()
        self.logger = logging.getLogger("BenchmarkRunner")
        self.logger.setLevel(logging.DEBUG)
        self.client = client
        self.output_file_path = output_file_path
        self.tavily_tool = tavily_tool
        self.model_name = model_name
        self.assistant = None
        try:
            self.result = json.load(open(output_file_path)) if os.path.exists(output_file_path) else []
        except:
            self.result = []

    def cleanup(self):
        for thread_id in self.thread_history:
            self.logger.info(f"delete thread {thread_id}")
            self.client.beta.threads.delete(thread_id=thread_id)
        if self.assistant:
            self.logger.info(f"delete assistant {self.assistant.id}")
            self.client.beta.assistants.delete(assistant_id=self.assistant.id)

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            self.cleanup()
        except Exception as e:
            self.logger.error(e)

    def run(self):
        self.logger.info(f"run started")
        for item in load_dataset("scholarly-shadows-syndicate/hotpotqa_with_qa_gpt35", split="validation", streaming=True).filter(lambda x: x["level"] == "hard"):
            self.logger.info(f"item id={item['id']}")   
            self.assistant = client.beta.assistants.create(
                name="benchmark-runner",
                model=self.model_name,
                tools=[{"type": "function", "function": convert_to_openai_function(self.tavily_tool)}]
            )
            self.logger.info(f"assistant id: {self.assistant.id}")
            
            run = self.client.beta.threads.create_and_run(
                assistant_id=self.assistant.id,
                thread={
                    "messages": [
                        {"role": "user", "content": item["question"]}
                    ]
                },
                stream=False)
            self.logger.info(f"run, id={run.id}, thread_id={run.thread_id}")

            self.thread_history.append(run.thread_id)
            result_item = {
                "ok": False,
                "answer": "",
                "truth": item["answer"], 
                "id": item["id"],
                "rt": 0
            }
            while True:
                ts_1 = time.time()
                run = self.client.beta.threads.runs.retrieve(thread_id=run.thread_id, run_id=run.id)
                if run.status == "queued" or run.status == "in_progress":
                    time.sleep(1)
                elif run.status == "requires_action":
                    tool_messages = []
                    for call in run.required_action.submit_tool_outputs.tool_calls:
                        self.logger.info(f"got tool call: {call.json()}")
                        if call.type == "function" and call.function.name == "tavily_search_results_json":
                            tool_result  = self.tavily_tool.invoke(call.function.arguments)
                            if isinstance(tool_result, list) and len(tool_result)>0 and isinstance(tool_result[0], dict):
                                combined_content = "\n".join([item["content"] for item in tool_result])
                                tool_messages.append({"tool_call_id": call.id, "output": combined_content})
                        else:
                            self.logger.error(f"Unknown tool call occurred, function name {call.function.name}")
                            break
                    if len(tool_messages) == len(run.required_action.submit_tool_outputs.tool_calls):
                        run = self.client.beta.threads.runs.submit_tool_outputs(thread_id=run.thread_id, run_id=run.id, tool_outputs=tool_messages)
                        self.logger.info(f"run object status after submit: {run.status}")
                    else:
                        self.logger.error("Not every call is responded.")
                        break
                elif run.status == "completed": 
                    messages = self.client.beta.threads.messages.list(thread_id=run.thread_id, order="asc")
                    result_item["ok"] = True
                    result_item["answer"] = messages.data[-1].content[0].text.value
                    self.logger.info("begin printing trajectory =============================")
                    for message in messages.data:
                        self.logger.info(f"{message.role}: {message.content[0].text.value}")
                    self.logger.info("finish printing trajectory =============================")
                    break
                else:
                    self.logger.error(f"run is in other terminal status: {run.to_json()}")
                    break    
            
            result_item["rt"] = time.time() - ts_1
            self.result.append(result_item)
            self.logger.info(f"id={result_item['id']}, ok={result_item['ok']}")
            
            # write down the result
            with open(self.output_file_path, "w") as output_json:
                json.dump(self.result, output_json)
        
            
    def get_metrics(self):
        with open(self.output_file_path, "r") as result_file:
            result = json.load(result_file)
            acc = np.average([compare_answer(item["answer"], item["truth"]) for item in result])
            rt_avg = np.average([item["rt"] for item in result])
            rt_std = np.std([item["rt"] for item in result])
            success_rate = np.average([1 if item["ok"] else 0 for item in result])
            
            logging.info(f"Success rate: {success_rate}")
            logging.info(f"Accuracy: {acc}")
            logging.info(f"Latency: {rt_avg} +/- {rt_std}")
            
            return success_rate, acc, rt_avg, rt_std
            

# Benchmarks


## With `mini-assistant`

Start mini assistant server.

* `llm_compiler` is used for agent execution
* `mixtral 7bx8` is hosted by vLLM. Please make sure you have set up `HUGGING_FACE_HUB_TOKEN` env for vLLM.

vLLM shell command using docker:

```shell
docker run --runtime nvidia --gpus all \
    -v /workspace/dropbox/huggingface_models:/root/.cache/huggingface \
    --env "HUGGING_FACE_HUB_TOKEN=${HUGGING_FACE_HUB_TOKEN}" \
    -p 8000:8000 \
    --ipc=host \
    vllm/vllm-openai:latest \
    --model TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ \
    --quantization marlin \
    --dtype=float16
```

mini-assistant shell command:

```shell
mkdir -p /tmp/mini-assistant-db
mkdir -p /tmp/mini-assistant-files
mini-assistant --db_file_path /tmp/assistant_eval.db \
  --file_store_path /tmp/mini-assistant-files \
  --agent_executor_type=llm_compiler \
  --model_provider=openai \
  --openai_port=8000 \
  --openai_host=192.168.0.134 \
  --openai_protocol=http \
  --port=9091 \
  --verbose
```

Please make sure to make necessary modification to `--openai_host`, `--openai_port` and `--openai_protocol` according to your own vLLM setup.  


And kick off benchmarks in python script:

In [85]:
if True:
    if not os.path.exists("./output"):
        os.mkdir("./output")
    client = OpenAI(base_url="http://localhost:9091/v1")
    with BenchmarkRunner(client=client, model_name="TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ", output_file_path="./output/miniassistant_result.json") as benchmark_runner:
        benchmark_runner.run()
        benchmark_runner.get_metrics()
    

INFO:BenchmarkRunner:run started
INFO:BenchmarkRunner:item id=5a8b57f25542995d1e6f1371
INFO:httpx:HTTP Request: POST http://localhost:9091/v1/assistants "HTTP/1.1 200 OK"
INFO:BenchmarkRunner:assistant id: asst_759510110350344192
INFO:httpx:HTTP Request: POST http://localhost:9091/v1/threads/runs "HTTP/1.1 200 OK"
INFO:BenchmarkRunner:run, id=run_759510110434230272, thread_id=thread_759510110434230273
INFO:httpx:HTTP Request: GET http://localhost:9091/v1/threads/thread_759510110434230273/runs/run_759510110434230272 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:9091/v1/threads/thread_759510110434230273/runs/run_759510110434230272 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:9091/v1/threads/thread_759510110434230273/runs/run_759510110434230272 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:9091/v1/threads/thread_759510110434230273/runs/run_759510110434230272 "HTTP/1.1 200 OK"
INFO:BenchmarkRunner:got tool call: {"id":"call_75951012146

KeyboardInterrupt: 

## With OpenAI's offering

Please make sure you have `OPENAI_API_KEY` setup in your environments.


In [73]:
if True:
    if not os.path.exists("./output"):
        os.mkdir("./output")
    client = OpenAI()
    with BenchmarkRunner(client=client, model_name="gpt-3.5-turbo", output_file_path="./output/openai_result.json") as benchmark_runner:
        # benchmark_runner.run()
        benchmark_runner.get_metrics()

INFO:root:Success rate: 0.3333333333333333
INFO:root:Accuracy: 0.0
INFO:root:Latency: 3.13826158841451 +/- 2.1034252851239037
